#### Métricas de las semillas

Al estar las muestras equilibradas para el entramiento de los modelos, la exactitud (accuracy), la precisión (precision), la sensibilidad (recall) y la puntuación F1 adquieren más relevancia y se vuelven más confiables para comparar el rendimiento de los modelos. 

El equilibrio de las clases reduce el sesgo hacia la clase mayoritaria, que es una de las principales limitaciones de estas métricas en conjuntos de datos desequilibrados.


In [ ]:
import pandas as pd
import os
import numpy as np
import datetime
import psutil
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt

from commons import myfunctions as myfunc

start_time = datetime.datetime.now()


Leer fichero con métricas de los modelos entrenados con semilla 42

In [ ]:

archivo1= os.path.join(myfunc.MET_DIR, "metricas_0042.csv")

fichero1 = os.path.basename(archivo1)
carpeta1 = os.path.dirname(archivo1)

df_tmp = myfunc.read_csv_to_df_spa(fichero1, carpeta1)[['tipo', 'select', 'clasific', 'accuracy', 'precision', 'recall', 'f1_score', 'roc_auc', 'roc_auc_ovr','fichero_modelo','indices_auc','indices_jaccard']]
df_tmp['tipo_select_clasific'] = df_tmp['tipo'] + '-' + df_tmp['select'] + '-' + df_tmp['clasific']
df_tmp['select_clasific'] = df_tmp['select'] + '-' + df_tmp['clasific']
df_tmp['auc'] = df_tmp['roc_auc'].fillna(0) + df_tmp['roc_auc_ovr'].fillna(0)

df_c = df_tmp.copy()

In [ ]:
def dibujar_grafico_barras(df1):
    df_g = df1.sort_values(by=["tipo_select_clasific","accuracy","precision"], ascending=True).copy().reset_index()

    ax = df_g.plot(kind='barh', x='tipo_select_clasific', figsize=(10, 8), width=0.9)
    ax.xaxis.set_major_locator(plt.MultipleLocator(0.1))  
    ax.grid(True, which='both', axis='x', linestyle='--', linewidth=0.5)

    # coger datos para dibujar una línea vertical que indique el valor mayor del accuracy
    max_accuracy = df_g['accuracy'].max()  
    max_accuracy_index = df_g['accuracy'].idxmax()  
    ax.axvline(x=max_accuracy, color='red', linestyle='--', linewidth=1) 

    for i, bar in enumerate(ax.patches):
        if i == max_accuracy_index:  
            ax.annotate(f'Max Accuracy: {max_accuracy:.4f}', 
                        xy=(max_accuracy, bar.get_y() + bar.get_height()/2),
                        xytext=(3, 0), textcoords='offset points',
                        ha='left', va='center')

    ax.set_xlabel('Métricas')
    ax.set_title('Comparación de métricas de diferentes algoritmos')
    ax.set_yticklabels(df_g['tipo_select_clasific'])
    ax.set_xlim(0, 1)

    ax.legend(title='Métricas', bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.tight_layout()
    plt.show()


In [ ]:
df1 = df_c.sort_values(["tipo","select","clasific"])
columnas = ["accuracy", "precision", "recall", "f1_score", "auc"]
df_suma = df1.groupby("tipo_select_clasific")[columnas].sum()
df_merge = df_suma.copy()


Métricas para los modelos de clasificación binaria con muestras reales

In [ ]:
myfunc.verbose("Valores de los modelos con el conjunto de datos bin_s, solo muestras reales")

df1 = df_merge[df_merge.index.str.startswith("bin_s")]

display(df1)


In [ ]:
dibujar_grafico_barras(df1)


Métricas para los modelos de clasificación binaria con muestras reales y sintéticas

In [ ]:
myfunc.verbose("Valores de los modelos con el conjunto de datos bin_m, muestras reales y sintéticas")

df1=df_merge[df_merge.index.str.startswith("bin_m")]
display(df1)

In [ ]:
dibujar_grafico_barras(df1)


Métricas para los modelos de clasificación multicáncer con muestras reales

In [ ]:
myfunc.verbose("Valores de los modelos con el conjunto de datos mul_s, solo muestras reales")

df1=df_merge[df_merge.index.str.startswith("mul_s")]
display(df1)

In [ ]:
dibujar_grafico_barras(df1)


Métricas para los modelos de clasificación multicáncer con muestras reales y sintéticas

In [ ]:
myfunc.verbose("Valores de los modelos con el conjunto de datos mul_s, muestras reales y sintéticas")

df1=df_merge[df_merge.index.str.startswith("mul_m")]
display(df1)

In [ ]:
dibujar_grafico_barras(df1)


#### Columnas que interesan y funciones de mínimos para las desviaciones y de máximos para el resto.

In [ ]:
columns_suma = ["accuracy", "precision", "recall", "f1_score", "auc"] 


def get_max_by_column(df):
  max_by_column = {}
  for column in df.columns:
    max_by_column[column] = df[column].idxmax(), df[column].max()

  return max_by_column


def get_min_by_column(df):
  min_by_column = {}
  for column in df.columns:
    min_by_column[column] = df[column].idxmin(), df[column].min()

  return min_by_column


def mostrar_mejores(tipo1, estadistico1, df1, orden_columnas=["accuracy"],ordenacion1=False):
  df_tipo = df1.query(f"tipo_select_clasific.str.startswith('{tipo1}')")
  if ordenacion1 == False:
    # búsqueda de la mayor media
    valores1 = pd.DataFrame(get_max_by_column(df_tipo))
  else:
    # búsqueda de la mínima desviación
    valores1 = pd.DataFrame(get_min_by_column(df_tipo))

  algoritmos = valores1.iloc[0].unique()
  columnas = df_tipo.columns
  df_tipo = df_tipo.round(4)
  
  myfunc.verbose(f"Algoritmos en primera posicion por alguna de sus columnas para conjunto de datos {tipo1} y estadístico {estadistico1}")

  display(df_tipo[df_tipo.index.isin(algoritmos)].sort_values(orden_columnas, ascending=ordenacion1))


#### Resumen de los mejores algoritmos para cada conjunto de datos

In [ ]:
orden_columnas=["accuracy", "precision", "recall"]
tipo1 = 'bin_s'
mostrar_mejores(tipo1, "media", df_merge[columns_suma], orden_columnas)
tipo1 = 'bin_m'
mostrar_mejores(tipo1, "media", df_merge[columns_suma], orden_columnas)
tipo1 = 'mul_s'
mostrar_mejores(tipo1, "media", df_merge[columns_suma], orden_columnas)
tipo1 = 'mul_m'
mostrar_mejores(tipo1, "media", df_merge[columns_suma], orden_columnas)


#### Finalización del notebook

In [ ]:
end_time = datetime.datetime.now()
total_time = end_time - start_time
myfunc.verbose(f"Notebook ha tardado {total_time} segundos")